In [1]:
import configparser
import psycopg2
from sql_queries import copy_table_queries, insert_table_queries

def load_staging_tables(cur, conn):
    """Copy S3 JSON data to staging_events_table and staging_songs_table \
    tables on database.

    Args:
        cur (cursor): Database connection object.
        conn (connection): Database connection object.

    """
    for query in copy_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

def insert_tables(cur, conn):
    """Gather data from staging tables to fact and dimensions tables 

    Args:
        cur (cursor): Database connection object.
        conn (connection): Database connection object.

    """
    for query in insert_table_queries:
        print(query)
        cur.execute(query)
        conn.commit()

In [2]:
def main():
    config = configparser.ConfigParser()
    config.read('dwh.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()


    copy staging_events_table from 's3://udacity-dend/log_data'
    credentials 'aws_iam_role=arn:aws:iam::952823547820:role/dwhRole'
    compupdate off region 'us-west-2'
    format as json 'auto'
    TIMEFORMAT 'epochmillisecs'


    copy staging_songs_table from 's3://udacity-dend/song_data/A/A/A/TRA'
    credentials 'aws_iam_role=arn:aws:iam::952823547820:role/dwhRole'
    compupdate off region 'us-west-2'
    format as json 'auto'

INSERT into songplay_tbl (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    SELECT
    ts, userid, level, song_id, artist_id, sessionid, location, useragent
    FROM
    (SELECT e.ts, e.userid, e.level, a.song_id, a.artist_id, e.sessionid, e.location, e.useragent
    FROM 
    staging_events_table e
    JOIN
    (select song.song_id, artist.artist_id, song.title, artist.name, song.duration
    FROM 
    song_tbl song
    JOIN 
    artist_tbl artist
    ON 
    song.artist_id = artist.artist_id) AS a
    ON
    (a.tit